In [1]:
import os
import re
from pathlib import Path
import pandas as pd
import numpy as np





In [6]:
import pandas as pd
import sys

# --- Configuration ---

# The CSV file you uploaded
INPUT_CSV_FILE = 'data/final_merged_fmri_t1_clinical (2).csv'

# The new Excel file we will create
OUTPUT_EXCEL_FILE = 'data/T1_first_session_data.xlsx'

# The one column you want to include in EVERY sheet
# Based on your request, this is 'Subject_Code'
KEY_COLUMN = 'Subject_Code'

# VVV --- THIS IS THE PART YOU EDIT --- VVV
#
# Define the sheets you want to create.
#
# - The 'key' is the name you want for the sheet (e.g., 'Clinical_Scores').
# - The 'value' is a list of all the *other* columns you want in that sheet.
#
# The KEY_COLUMN ('Subject_Code') will be added for you automatically.
# I've added some examples based on your CSV's column names.
#
SHEETS_TO_CREATE = {
    'global_parameters': [
    "brainsegvol",
    "cerebralwhitemattervol",
    "cortexvol",
    "etiv",
    "lhcortexvol",
    "rhcortexvol",
    "subcortgrayvol",
    "totalgrayvol",
    "meanthickness_lh",
    "meanthickness_rh",
    "whitesurfarea_lh",
    "whitesurfarea_rh"],

    'subcortical_vol': [
    "3rd_ventricle",
    "4th_ventricle",
    "5th_ventricle",
    "brain_stem",
    "cc_anterior",
    "cc_central",
    "cc_mid_anterior",
    "cc_mid_posterior",
    "cc_posterior",
    "csf",
    "left_accumbens_area",
    "left_amygdala",
    "left_caudate",
    "left_cerebellum_cortex",
    "left_cerebellum_white_matter",
    "left_choroid_plexus",
    "left_hippocampus",
    "left_inf_lat_vent",
    "left_lateral_ventricle",
    "left_non_wm_hypointensities",
    "left_pallidum",
    "left_putamen",
    "left_thalamus",
    "left_ventraldc",
    "left_vessel",
    "left_wm_hypointensities",
    "non_wm_hypointensities",
    "optic_chiasm",
    "right_accumbens_area",
    "right_amygdala",
    "right_caudate",
    "right_cerebellum_cortex",
    "right_cerebellum_white_matter",
    "right_choroid_plexus",
    "right_hippocampus",
    "right_inf_lat_vent",
    "right_lateral_ventricle",
    "right_non_wm_hypointensities",
    "right_pallidum",
    "right_putamen",
    "right_thalamus",
    "right_ventraldc",
    "right_vessel",
    "right_wm_hypointensities",
    "wm_hypointensities"
],

    "cortical_vol_lr": [
    "bankssts_lh",
    "bankssts_rh",
    "caudalanteriorcingulate_lh",
    "caudalanteriorcingulate_rh",
    "caudalmiddlefrontal_lh",
    "caudalmiddlefrontal_rh",
    "cuneus_lh",
    "cuneus_rh",
    "entorhinal_lh",
    "entorhinal_rh",
    "frontalpole_lh",
    "frontalpole_rh",
    "fusiform_lh",
    "fusiform_rh",
    "inferiorparietal_lh",
    "inferiorparietal_rh",
    "inferiortemporal_lh",
    "inferiortemporal_rh",
    "insula_lh",
    "insula_rh",
    "isthmuscingulate_lh",
    "isthmuscingulate_rh",
    "lateraloccipital_lh",
    "lateraloccipital_rh",
    "lateralorbitofrontal_lh",
    "lateralorbitofrontal_rh",
    "lingual_lh",
    "lingual_rh",
    "medialorbitofrontal_lh",
    "medialorbitofrontal_rh",
    "middletemporal_lh",
    "middletemporal_rh",
    "paracentral_lh",
    "paracentral_rh",
    "parahippocampal_lh",
    "parahippocampal_rh",
    "parsopercularis_lh",
    "parsopercularis_rh",
    "parsorbitalis_lh",
    "parsorbitalis_rh",
    "parstriangularis_lh",
    "parstriangularis_rh",
    "pericalcarine_lh",
    "pericalcarine_rh",
    "postcentral_lh",
    "postcentral_rh",
    "posteriorcingulate_lh",
    "posteriorcingulate_rh",
    "precentral_lh",
    "precentral_rh",
    "precuneus_lh",
    "precuneus_rh",
    "rostralanteriorcingulate_lh",
    "rostralanteriorcingulate_rh",
    "rostralmiddlefrontal_lh",
    "rostralmiddlefrontal_rh",
    "superiorfrontal_lh",
    "superiorfrontal_rh",
    "superiorparietal_lh",
    "superiorparietal_rh",
    "superiortemporal_lh",
    "superiortemporal_rh",
    "supramarginal_lh",
    "supramarginal_rh",
    "temporalpole_lh",
    "temporalpole_rh",
    "transversetemporal_lh",
    "transversetemporal_rh"
]
,
    'cortical_lobes_lr': [
    "Cingulate_lh",
    "Cingulate_rh",
    "Frontal_lh",
    "Frontal_rh",
    "Insula_lh",
    "Insula_rh",
    "Occipital_lh",
    "Occipital_rh",
    "Parietal_lh",
    "Parietal_rh",
    "Temporal_lh",
    "Temporal_rh"
],


    'cortical_lobes_full_brain': [
    "cingulate",
    "frontal",
    "insula",
    "occipital",
    "parietal",
    "temporal"
],
    'cortical_networks_full_brain': [
    "DMN_GM",
    "DA_GM",
    "FPN_GM",
    "LIM_GM",
    "SMN_GM",
    "VAN_GM",
    "VIS_GM"
]

}
# ^^^ --- THIS IS THE PART YOU EDIT --- ^^^




In [8]:
"""
Main function to read CSV and write to a multi-sheet Excel file.
"""
try:
    # 1. Load the entire CSV into a pandas DataFrame
    print(f"Reading input file: {INPUT_CSV_FILE}...")
    main_df = pd.read_csv(INPUT_CSV_FILE)
    print("CSV file loaded successfully.")

    # Build a case-insensitive mapping: lower_name -> original_name
    col_map = {c.lower(): c for c in main_df.columns}

    # 2. Check that the key column exists (case-insensitive)
    key_lower = KEY_COLUMN.lower()
    if key_lower not in col_map:
        print(f"Error: The key column '{KEY_COLUMN}' was not found in the CSV (case-insensitive search).")
        print("Please check the KEY_COLUMN variable.")
        print(f"Available columns are: {main_df.columns.tolist()}")
        sys.exit(1)

    key_actual = col_map[key_lower]  # actual column name in the DataFrame

    # 3. Create an ExcelWriter object to handle writing to multiple sheets
    with pd.ExcelWriter(OUTPUT_EXCEL_FILE, engine='openpyxl') as writer:
        print(f"Creating Excel file: {OUTPUT_EXCEL_FILE}...")

        # 4. Loop through your configuration
        for sheet_name, column_list in SHEETS_TO_CREATE.items():

            # 5. Build the final list of columns for this sheet
            final_columns = []

            # Check if key column is already in the list (case-insensitive)
            column_list_lower = [c.lower() for c in column_list]
            if key_lower not in column_list_lower:
                final_columns.append(KEY_COLUMN)   # we'll resolve to real name below

            final_columns.extend(column_list)

            # 6. Check for missing columns and create the sheet DataFrame
            valid_columns = []
            seen_lowers = set()  # to avoid duplicates with different cases

            for col in final_columns:
                col_lower = col.lower()
                if col_lower in seen_lowers:
                    continue  # skip duplicates
                if col_lower in col_map:
                    valid_columns.append(col_map[col_lower])  # use actual name from CSV
                    seen_lowers.add(col_lower)
                else:
                    print(f"  - Warning: Column '{col}' (for sheet '{sheet_name}') "
                          f"not found in CSV (case-insensitive). It will be skipped.")

            if not valid_columns:
                print(f"  - Error: No valid columns found for sheet '{sheet_name}'. "
                      f"This sheet will be empty or skipped.")
                continue

            # 7. Create the new DataFrame for this sheet
            sheet_df = main_df[valid_columns]

            # 8. Write this DataFrame to the Excel file
            sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)
            print(f"  -> Successfully created sheet: '{sheet_name}' with {len(valid_columns)} columns.")

    print(f"\nAll done! Your file '{OUTPUT_EXCEL_FILE}' has been created.")

except FileNotFoundError:
    print(f"Error: The file '{INPUT_CSV_FILE}' was not found.")
    print("Please make sure the CSV file is in the same directory as this script.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Reading input file: data/final_merged_fmri_t1_clinical (2).csv...
CSV file loaded successfully.
Creating Excel file: data/T1_first_session_data.xlsx...
  - Warning: Column 'brainsegvol' (for sheet 'global_parameters') not found in CSV (case-insensitive). It will be skipped.
  -> Successfully created sheet: 'global_parameters' with 12 columns.
  - Warning: Column '3rd_ventricle' (for sheet 'subcortical_vol') not found in CSV (case-insensitive). It will be skipped.
  - Warning: Column '4th_ventricle' (for sheet 'subcortical_vol') not found in CSV (case-insensitive). It will be skipped.
  - Warning: Column '5th_ventricle' (for sheet 'subcortical_vol') not found in CSV (case-insensitive). It will be skipped.
  - Warning: Column 'brain_stem' (for sheet 'subcortical_vol') not found in CSV (case-insensitive). It will be skipped.
  - Warning: Column 'cc_anterior' (for sheet 'subcortical_vol') not found in CSV (case-insensitive). It will be skipped.
  - Warning: Column 'cc_central' (for sheet '

In [3]:
def collect_files(folder: Path, recursive: bool = False):
    if recursive:
        return [p for p in folder.rglob("*.xlsx") if p.is_file()]
    return [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() == ".xlsx"]


In [4]:

def list_excel_files(folder: Path, recursive: bool = False):
    if recursive:
        return sorted([p for p in folder.rglob("*.xlsx") if p.is_file()])
    return sorted([p for p in folder.iterdir() if p.is_file() and p.suffix.lower() == ".xlsx"])



In [5]:
def pick_column(df: pd.DataFrame, candidates):
    """Return the first matching column name in df (case-insensitive), else None."""
    cols_map = {c.lower(): c for c in df.columns}
    for c in candidates:
        if c.lower() in cols_map:
            return cols_map[c.lower()]
    # try partial matches (e.g., "grayvol" inside "grayvol (mm3)")
    for c in candidates:
        for k, v in cols_map.items():
            if c.lower() in k:
                return v
    return None


In [6]:
def slugify(name: str) -> str:
    """lowercase + replace non-alnum with underscores + collapse repeats."""
    s = re.sub(r"\W+", "_", str(name).strip().lower())
    s = re.sub(r"_+", "_", s).strip("_")
    return s

In [7]:
# Create a stable, pretty column order:
#   subject_code first, then all lh/rh ses1 followed by ses2 (alphabetical by struct)
def col_sort_key(c):
    # subject_code stays first
    if c == "subject_code":
        return (0, "", "", 0)
    # parse pattern: <struct>_<hemi>_ses<1|2>
    m = re.match(r"^(.*)_(lh|rh)_ses([12])$", c)
    if m:
        struct, hemi, ses = m.group(1), m.group(2), int(m.group(3))
        return (1, struct, hemi, ses)
    # anything unexpected goes to the end
    return (2, c, "", 99)

In [8]:
def merge_two_cols(dframe, cols, how="sum"):
    # cols is a list of existing columns (1 or 2). We want:
    # - NaN if both missing
    # - sum/mean ignoring NaNs otherwise
    cols = [c for c in cols if c is not None and c in dframe.columns]
    if not cols:
        return pd.Series(np.nan, index=dframe.index)
    if how == "mean":
        return dframe[cols].mean(axis=1)
    # default sum
    return dframe[cols].sum(axis=1, min_count=1)


In [9]:
def reduce_cols(dframe, cols, how="sum"):
    cols = [c for c in cols if c in dframe.columns]
    if not cols:
        return pd.Series(np.nan, index=dframe.index)
    return dframe[cols].sum(axis=1, min_count=1)

In [10]:
# --- 4) Order columns nicely: subject_code, then lobe alpha, ses1 before ses2, lh before rh
def order_key_lobe(c):
    if c == "subject_code":
        return (0, "", 0, "")
    m = re.match(r"^(?P<lobe>.+)_(?P<hemi>lh|rh)_ses(?P<ses>\d+)$", c)
    if m:
        lobe = m.group("lobe")
        hemi = m.group("hemi")
        ses  = int(m.group("ses"))
        hemi_order = 0 if hemi == "lh" else 1
        return (1, lobe, ses, hemi_order)
    return (2, c, 99, 99)


In [11]:
def _read_csv(path: Path, merge_key: str) -> pd.DataFrame:
    if not path.exists():
        print(f"WARNING: file not found -> {path}")
        return pd.DataFrame(columns=[merge_key])
    df = pd.read_csv(path)
    # normalize key dtype for safe merging/sorting
    if merge_key in df.columns:
        df[merge_key] = df[merge_key].astype(str)
    return df



In [12]:
def _order_columns_subject_first(df: pd.DataFrame, key: str = "subject_code") -> pd.DataFrame:
    cols = list(df.columns)
    if key in cols:
        return df[[key] + [c for c in cols if c != key]]
    return df

In [13]:
def _sort_consistent(df: pd.DataFrame, key: str = "subject_code") -> pd.DataFrame:
    """Alphabetical by key; if numbers exist, use them as a secondary natural sort."""
    if key not in df.columns:
        return df
    key_series = df[key].astype(str)
    # Extract first number if present; fall back to NaN
    num = key_series.str.extract(r"(\d+)")[0].astype(float)
    # Primary: alphabetical by key; Secondary: numeric ascending (NaN last)
    return (df.assign(_num=num)
              .sort_values(by=[key, "_num"], kind="mergesort")
              .drop(columns="_num"))


In [14]:
def _autosize_columns(writer: pd.ExcelWriter, df: pd.DataFrame, sheet_name: str, max_width: int = 60):
    """Best-effort autosize; harmless no-op if engine doesn't support."""
    try:
        ws = writer.sheets[sheet_name]
        for idx, col in enumerate(df.columns, 1):
            s = df[col].astype(str)
            longest = max([len(str(col))] + [len(x) for x in s.tolist()] + [10])
            ws.column_dimensions[ws.cell(row=1, column=idx).column_letter].width = min(longest + 2, max_width)
    except Exception:
        pass

In [15]:
records = []
sheet_names_ref = None
# --- CONFIG --- #
FOLDER = Path("longitude_stats_without_normalizations")  # change to your folder
SUMMARY_CSV = Path("longitude_stats_without_normalizations/2_sess_T1_summary.csv")  # change if you want
RECURSIVE = False  # set True if files are in subfolders
# --------------- #


In [16]:
# Matches: 002_NT_ses-1_merged.xlsx  or  015_CT_ses-2_merged.xlsx  (case-insensitive)
file_regex = re.compile(
    r"(?P<subject>\d{3})_(?P<group>CT|NT)_ses-(?P<session>[12])_merged\.xlsx$",
    re.IGNORECASE,
)

for path in sorted(collect_files(FOLDER, RECURSIVE)):
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    subject_code = f"{group}{subject}"

    # Capture sheet names from the first valid workbook
    if sheet_names_ref is None:
        try:
            xls = pd.ExcelFile(path)
            sheet_names_ref = list(xls.sheet_names)
        except Exception as e:
            print(f"Warning: failed to read sheet names from '{path}': {e}")
            sheet_names_ref = []


In [17]:

# If we didn't find any matching files, still create a CSV with just the first two columns
if sheet_names_ref is None:
    sheet_names_ref = []

# Build dataframe with empty columns for each sheet name
columns = ["subject_code"] + sheet_names_ref
df = pd.DataFrame(records, columns=columns)

# Ensure those sheet columns are empty strings (not NaN)
for s in sheet_names_ref:
    df[s] = ""

# Nice sorting: group (CT/NT) + numeric subject + session
if not df.empty:
    df["_grp"] = df["subject_code"].str.extract(r"^(CT|NT)")
    df["_num"] = df["subject_code"].str.extract(r"(\d{3})").astype(int)
    df = df.sort_values(by=["_grp", "_num"]).drop(columns=["_grp", "_num"]).reset_index(drop=True)

# Save CSV
SUMMARY_CSV.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(SUMMARY_CSV, index=False)

print(f"CSV written to: {SUMMARY_CSV}")
display(df.head(10))

CSV written to: longitude_stats_without_normalizations\2_sess_T1_summary.csv


,subject_code,session,aparc_summary,aparc_voxels,aseg_summary,aseg_voxels,wmparc_summary,wmparc_voxels
0,CT003,1,,,,,,
1,CT003,2,,,,,,
2,CT007,1,,,,,,
3,CT007,2,,,,,,
4,CT010,1,,,,,,
5,CT010,2,,,,,,
6,CT012,1,,,,,,
7,CT012,2,,,,,,
8,CT013,1,,,,,,
9,CT013,2,,,,,,


In [18]:

# ---------- CONFIG ----------
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_lh_rh.csv")  # output file
SHEET_NAME = "aparc_voxels"  # sheet to read
RECURSIVE = False            # set True if files are in subfolders
# ----------------------------


# Accumulate rows keyed by subject_code
rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Locate columns (robust to naming/case variants)
    col_struct = pick_column(df, ["StructName"])
    col_hemi   = pick_column(df, ["Hemisphere"])
    col_gray   = pick_column(df, ["GrayVol"])

    if not all([col_struct, col_hemi, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"Found StructName = {col_struct}, Hemisphere = {col_hemi}, GrayVol = {col_gray}")
        continue

    # Initialize subject row if needed
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Build columns like insula_lh_ses1 with values from grayvol
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        hemi = slugify(row[col_hemi])  # expects 'lh'/'rh'
        # Normalize common hemisphere values to 'lh'/'rh'
        if hemi in ("left", "l"): hemi = "lh"
        if hemi in ("right", "r"): hemi = "rh"

        # skip rows missing essentials
        if not struct or hemi not in ("lh", "rh"):
            continue

        col_name = f"{struct}_{hemi}_ses{session}"
        val = row[col_gray]
        # Coerce to numeric (keep NaN if not coercible)
        try:
            val = pd.to_numeric(val)
        except Exception:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)
    subject_code = f"{group}{subject}"      # e.g., CT002
    mark_extraction_complete(subject_code, sheet_name="aparc_voxels")# e.g., CT002 / NT015


all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build DataFrame
df_out = pd.DataFrame(rows_by_subject.values())
# Ensure all expected columns are present
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan

# Reorder and keep subject_code first
df_out = df_out[all_columns]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))

Saved: longitude_stats_without_normalizations\2_sessions_aparc_voxels_lh_rh.csv


,subject_code,bankssts_lh_ses1,bankssts_lh_ses2,bankssts_rh_ses1,bankssts_rh_ses2,caudalanteriorcingulate_lh_ses1,caudalanteriorcingulate_lh_ses2,caudalanteriorcingulate_rh_ses1,caudalanteriorcingulate_rh_ses2,caudalmiddlefrontal_lh_ses1,...,supramarginal_rh_ses1,supramarginal_rh_ses2,temporalpole_lh_ses1,temporalpole_lh_ses2,temporalpole_rh_ses1,temporalpole_rh_ses2,transversetemporal_lh_ses1,transversetemporal_lh_ses2,transversetemporal_rh_ses1,transversetemporal_rh_ses2
0,NT002,2670,2582,1960,1965,1423,1397,2060,1979,5850,...,10638,10174,2716,2393,2528,2486,1047,1018,861,823
1,CT003,2713,2673,2701,2613,1627,1683,2014,2015,7493,...,10018,9780,3372,3350,3198,3063,1012,983,769,757
2,NT005,2788,2703,2196,2144,1074,1025,2215,2176,4278,...,10830,9942,2332,2312,2576,2633,1243,1216,881,850
3,CT007,1986,2015,2242,2177,1744,1785,2951,3048,5069,...,9359,9219,2506,2852,2247,2490,1272,1227,899,929
4,CT010,2542,2543,2294,2257,1463,1418,2369,2361,5473,...,7021,7075,2707,2665,2348,2295,1038,1032,811,800


In [19]:


# ---------- CONFIG ----------
FOLDER = Path("longitude_stats_without_normalizations")  # change to your folder
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aseg_data.csv")  # output file
SHEET_NAME = "aseg_voxels"  # <-- aseg sheet
RECURSIVE = False            # set True if files are in subfolders
# ----------------------------




# Accumulate rows keyed by subject_code
rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    subject_code = f"{group}{subject}"  # e.g., CT002 / NT015
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Locate columns (robust to naming/case variants)
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"Found struct={col_struct}, vol={col_gray}")
        continue

    # Initialize subject row if needed
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Build columns like <struct>_ses1 with values from grayvol
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        col_name = f"{struct}_ses{session}"
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except Exception:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)
    subject_code = f"{group}{subject}"      # e.g., CT002
    session = int(m.group("session"))       # 1 or 2
    mark_extraction_complete(subject_code, session, sheet_name="aseg_voxels")# e.g., CT002 / NT015


all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build DataFrame
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


Saved: longitude_stats_without_normalizations\2_sessions_aseg_data.csv


,subject_code,3rd_ventricle_ses1,3rd_ventricle_ses2,4th_ventricle_ses1,4th_ventricle_ses2,5th_ventricle_ses1,5th_ventricle_ses2,brain_stem_ses1,brain_stem_ses2,cc_anterior_ses1,...,right_thalamus_ses1,right_thalamus_ses2,right_ventraldc_ses1,right_ventraldc_ses2,right_vessel_ses1,right_vessel_ses2,right_wm_hypointensities_ses1,right_wm_hypointensities_ses2,wm_hypointensities_ses1,wm_hypointensities_ses2
0,NT002,922.3,869.1,1157.7,1066.4,0.0,0.0,19800.5,20048.8,672.9,...,7168.8,7047.9,4098.9,4175.0,69.3,69.5,0.0,0.0,667.7,609.2
1,CT003,867.7,894.9,1899.4,1816.2,4.5,2.7,20241.3,20001.1,829.5,...,7814.8,7764.3,3942.3,3987.4,131.3,122.0,0.0,0.0,944.5,852.4
2,NT005,847.5,859.8,1535.5,1714.9,26.1,19.2,20201.7,19714.8,1036.0,...,7178.5,6962.1,3766.0,3540.1,63.0,82.7,0.0,0.0,360.6,305.4
3,CT007,792.9,813.4,2136.5,2081.8,10.6,9.7,22519.3,22830.1,1030.1,...,8271.4,8394.8,4074.8,4190.7,82.8,100.4,0.0,0.0,1351.4,975.4
4,CT010,1063.8,1008.2,2666.3,2634.2,0.0,0.0,22630.1,22641.8,1047.6,...,8394.3,8186.3,4193.2,4172.2,79.4,70.1,0.0,0.0,311.7,377.4


sdewew

# ---------- CONFIG ----------
FOLDER = Path("longitude_stats_without_normalizations")  # change to your folder
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_wmparc_data.csv")  # output file
SHEET_NAME = "wmparc_voxels"  # <-- aseg sheet
RECURSIVE = False            # set True if files are in subfolders
# ----------------------------




# Accumulate rows keyed by subject_code
rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, RECURSIVE)
for path in files:
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))

    subject_code = f"{group}{subject}"  # e.g., CT002 / NT015
    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    # Locate columns (robust to naming/case variants)
    col_struct = pick_column(df, ["StructName"])
    col_gray   = pick_column(df, ["Volume_mm3"])

    if not all([col_struct, col_gray]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"Found struct={col_struct}, vol={col_gray}")
        continue

    # Initialize subject row if needed
    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Build columns like <struct>_ses1 with values from grayvol
    for _, row in df.iterrows():
        struct = slugify(row[col_struct])
        if not struct:
            continue

        col_name = f"{struct}_ses{session}"
        val = row[col_gray]
        try:
            val = pd.to_numeric(val)
        except Exception:
            val = np.nan

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)
    subject_code = f"{group}{subject}"      # e.g., CT002
    session = int(m.group("session"))       # 1 or 2
    mark_extraction_complete(subject_code, session, sheet_name="wmparc_voxels")# e.g., CT002 / NT015


all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)

# Build DataFrame
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(5))


In [21]:
# Build per-subject table from aparc_summary, including hemisphere in column names.
# Requires helpers already defined: file_regex, list_excel_files, slugify, pick_column, col_sort_key, mark_extraction_complete

from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
FOLDER = Path("longitude_stats_without_normalizations")
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_summary_data.csv")
SHEET_NAME = "aparc_summary"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]
HEMI_COLUMN_CANDIDATES      = ["Hemisphere"]

# Keep ONLY these ShortName rows (exact matches as they appear in the sheet):
SHORTNAMES_TO_KEEP = ["WhiteSurfArea","MeanThickness"]
    # e.g. "insula", "precentral", "bankssts", ...


# Optional renames for output columns (after slugify)
RENAMES = {}
# ----------------------------

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))
    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)
    col_hemi  = pick_column(df, HEMI_COLUMN_CANDIDATES)

    if not all([col_short, col_val, col_hemi]):
        print(f"Warning: missing needed columns in {path.name}. "
              f"Found ShortName={col_short}, Value={col_val}, Hemisphere={col_hemi}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Filter only desired ShortNames (exact match on the raw values)
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)].copy()

    for _, r in sub.iterrows():
        raw_short = str(r[col_short])
        hemi_raw  = str(r[col_hemi]).strip().lower()

        # normalize hemisphere to lh/rh
        if hemi_raw in ("left", "l"):  hemi = "lh"
        elif hemi_raw in ("right", "r"): hemi = "rh"
        elif hemi_raw in ("lh", "rh"): hemi = hemi_raw
        else:
            # skip rows without a valid hemisphere
            continue

        key = slugify(raw_short)
        key = RENAMES.get(key, key)

        col_name = f"{key}_{hemi}_ses{session}"

        # coerce Value to numeric
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # mark summary done for this subject/session/sheet
    mark_extraction_complete(subject_code, session, sheet_name=SHEET_NAME)

# finalize dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: longitude_stats_without_normalizations\2_sessions_aparc_summary_data.csv


,subject_code,meanthickness_lh_ses1,meanthickness_lh_ses2,meanthickness_rh_ses1,meanthickness_rh_ses2,whitesurfarea_lh_ses1,whitesurfarea_lh_ses2,whitesurfarea_rh_ses1,whitesurfarea_rh_ses2
0,NT002,2.57432,2.49454,2.51656,2.44400,84456.7,83960.6,85798.8,84895.4
1,CT003,2.63823,2.62996,2.65913,2.62355,90104.7,89147.2,88973.8,88186.2
2,NT005,2.65105,2.58844,2.62182,2.58579,79621.9,76358.0,78921.6,76241.0
3,CT007,2.48926,2.54471,2.51000,2.54911,87803.7,87857.1,87253.2,86583.0
4,CT010,2.59687,2.55895,2.57176,2.53221,83175.4,82461.1,82291.4,82149.6
5,CT012,2.67354,2.65185,2.66016,2.65112,83156.0,82023.9,81731.3,81021.9
6,CT013,2.60889,2.61030,2.61542,2.58230,89042.4,86343.2,88615.8,85778.4
7,NT015,2.61509,2.60183,2.61451,2.57076,78292.3,78024.0,77973.0,78038.7
8,NT016,2.61350,2.58100,2.62777,2.61197,86003.8,83855.6,85369.8,83788.8
9,NT017,2.55446,2.51326,2.59662,2.52521,86593.2,82282.6,85102.8,80878.6


In [22]:


# ---------- CONFIG ----------
FOLDER = Path("longitude_stats_without_normalizations")
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aseg_summary_data.csv")
SHEET_NAME = "aseg_summary"   # e.g., "some_metrics_sheet"

SHORTNAME_COLUMN_CANDIDATES = ["ShortName"]
VALUE_COLUMN_CANDIDATES     = ["Value"]

# Keep ONLY these ShortName rows (exact matches as they appear in the sheet):
SHORTNAMES_TO_KEEP = ["eTIV", "BrainSegVol","lhCortexVol","rhCortexVol","CortexVol","CerebralWhiteMatterVol",'SubCortGrayVol',"TotalGrayVol"]

rows_by_subject = {}
all_feature_cols = set()

files = list_excel_files(FOLDER, recursive=False)
for path in files:
    m = file_regex.match(path.name)
    if not m:
        continue

    subject = m.group("subject")
    group = m.group("group").upper()
    session = int(m.group("session"))
    subject_code = f"{group}{subject}"

    try:
        df = pd.read_excel(path, sheet_name=SHEET_NAME)
    except Exception as e:
        print(f"Warning: couldn't read sheet '{SHEET_NAME}' in {path.name}: {e}")
        continue

    col_short = pick_column(df, SHORTNAME_COLUMN_CANDIDATES)
    col_val   = pick_column(df, VALUE_COLUMN_CANDIDATES)
    if not all([col_short, col_val]):
        print(f"Warning: missing needed columns in {path.name}. Found ShortName={col_short}, Value={col_val}")
        continue

    if subject_code not in rows_by_subject:
        rows_by_subject[subject_code] = {"subject_code": subject_code}

    # Filter rows to the desired ShortNames
    sub = df[df[col_short].astype(str).isin(SHORTNAMES_TO_KEEP)].copy()

    for _, r in sub.iterrows():
        raw_short = str(r[col_short])
        key = slugify(raw_short)
        key = RENAMES.get(key, key)  # optional rename
        col_name = f"{key}_ses{session}"

        # coerce Value to numeric
        val = pd.to_numeric(r[col_val], errors="coerce")

        rows_by_subject[subject_code][col_name] = val
        all_feature_cols.add(col_name)

    # mark summary done for this subject/session/sheet
    mark_extraction_complete(subject_code, session, sheet_name=SHEET_NAME)

# finalize dataframe
all_columns = ["subject_code"] + sorted(all_feature_cols, key=col_sort_key)
df_out = pd.DataFrame(rows_by_subject.values())
for c in all_columns:
    if c not in df_out.columns:
        df_out[c] = np.nan
df_out = df_out[all_columns]

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
df_out.to_csv(OUT_CSV, index=False)

print(f"Saved: {OUT_CSV}")
display(df_out.head(10))


Saved: longitude_stats_without_normalizations\2_sessions_aseg_summary_data.csv


,subject_code,brainsegvol_ses1,brainsegvol_ses2,cerebralwhitemattervol_ses1,cerebralwhitemattervol_ses2,cortexvol_ses1,cortexvol_ses2,etiv_ses1,etiv_ses2,lhcortexvol_ses1,lhcortexvol_ses2,rhcortexvol_ses1,rhcortexvol_ses2,subcortgrayvol_ses1,subcortgrayvol_ses2,totalgrayvol_ses1,totalgrayvol_ses2
0,NT002,1110228.0,1088457.0,419457.0,416588.0,490608.236876,471911.310674,1.422479e+06,1.422479e+06,245669.103788,237397.422763,244939.133088,234513.887911,58014.0,57402.0,655161.236876,636146.310674
1,CT003,1184745.0,1168965.0,458291.0,456157.0,531916.401250,519861.644142,1.503435e+06,1.503435e+06,266407.973214,260860.911186,265508.428036,259000.732956,59295.0,58931.0,689043.401250,675766.644142
2,NT005,1110667.0,1067155.0,423652.0,406878.0,471502.137855,443915.716838,1.415292e+06,1.415292e+06,237566.663194,222745.097278,233935.474661,221170.619560,56161.0,54908.0,641680.137855,613641.716838
3,CT007,1138151.0,1149703.0,421712.0,425634.0,489541.405657,496968.257306,1.431403e+06,1.431403e+06,243929.482988,249904.719304,245611.922669,247063.538002,63658.0,64002.0,673969.405657,681836.257306
4,CT010,1122091.0,1115262.0,408536.0,412789.0,474689.564570,466056.025773,1.439088e+06,1.439088e+06,239267.805546,234389.921100,235421.759024,231666.104673,59937.0,59739.0,657339.564570,648058.025773
5,CT012,1107827.0,1091554.0,391761.0,389121.0,497944.071008,486970.812681,1.453701e+06,1.453701e+06,251339.320238,244323.770697,246604.750770,242647.041984,62131.0,61648.0,668019.071008,654484.812681
6,CT013,1131618.0,1085518.0,400730.0,382679.0,524760.488220,498031.891560,1.419949e+06,1.419949e+06,262316.073827,250895.428303,262444.414393,247136.463257,62748.0,60772.0,692313.488220,666431.891560
7,NT015,1037379.0,1038595.0,377454.0,386570.0,458988.573462,453851.341338,1.331148e+06,1.331148e+06,229765.717092,228380.735128,229222.856370,225470.606209,57936.0,57469.0,625411.573462,617026.341338
8,NT016,1143170.0,1110370.0,405225.0,394230.0,504111.022004,484988.825810,1.267231e+06,1.267231e+06,252366.166819,241518.612710,251744.855185,243470.213100,59494.0,57989.0,694629.022004,672629.825810
9,NT017,1153453.0,1078295.0,436138.0,408880.0,497094.953205,450596.126581,1.456538e+06,1.456538e+06,248767.239049,227148.873162,248327.714156,223447.253419,57825.0,55245.0,664121.953205,616935.126581


In [23]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ---------- CONFIG ----------
IN_CSV  = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_lh_rh.csv")  # or your lh/rh file
OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_merge.csv")
AGGREGATION = "sum"   # options: "sum" (default) or "mean"
# ----------------------------

# Load
df = pd.read_csv(IN_CSV)

# Regex to catch columns like: insula_lh_ses1, bankssts_rh_ses2, etc.
pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)_ses(?P<ses>[12])$", re.IGNORECASE)

# Map (struct, session) -> list of columns [lh_col, rh_col] (whichever exist)
pairs = {}
other_cols = []  # cols we keep as-is (e.g., subject_code)
for col in df.columns:
    m = pat.match(col)
    if m:
        struct = m.group("struct")
        hemi = m.group("hemi").lower()
        ses = int(m.group("ses"))
        key = (struct, ses)
        pairs.setdefault(key, {"lh": None, "rh": None})
        pairs[key][hemi] = col
    else:
        other_cols.append(col)

# Build merged dataframe
out = df[other_cols].copy()  # keep subject_code and any non-lh/rh columns

# Create merged columns like "<struct>_ses1" / "<struct>_ses2"
for (struct, ses), hemis in pairs.items():
    new_col = f"{struct}_ses{ses}"
    out[new_col] = merge_two_cols(df, [hemis.get("lh"), hemis.get("rh")], how=AGGREGATION)

# Optional: drop any original lh/rh columns entirely (keep only merged + other)
# If you want to *also* keep the original columns, comment the next two lines.
lh_rh_cols = [c for c in df.columns if pat.match(c)]
out = out[[c for c in out.columns if c not in lh_rh_cols]]



ordered_cols = sorted(out.columns, key=col_sort_key)
out = out[ordered_cols]

# Save
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved merged file: {OUT_CSV}")
display(out.head(10))


Saved merged file: longitude_stats_without_normalizations\2_sessions_aparc_voxels_merge.csv


,subject_code,bankssts_ses1,bankssts_ses2,caudalanteriorcingulate_ses1,caudalanteriorcingulate_ses2,caudalmiddlefrontal_ses1,caudalmiddlefrontal_ses2,cuneus_ses1,cuneus_ses2,entorhinal_ses1,...,superiorparietal_ses1,superiorparietal_ses2,superiortemporal_ses1,superiortemporal_ses2,supramarginal_ses1,supramarginal_ses2,temporalpole_ses1,temporalpole_ses2,transversetemporal_ses1,transversetemporal_ses2
0,NT002,4630,4547,3483,3376,12457,11686,8233,8022,3885,...,29238,28032,24431,23930,21061,20247,5244,4879,1908,1841
1,CT003,5414,5286,3641,3698,13923,13379,6541,6480,3679,...,32101,31242,24854,24200,19754,19265,6570,6413,1781,1740
2,NT005,4984,4847,3289,3201,9134,8261,5644,5318,4049,...,28052,25215,23734,22924,20774,19162,4908,4945,2124,2066
3,CT007,4228,4192,4695,4833,11265,11605,5775,5824,3691,...,25890,26348,28656,28823,18711,18882,4753,5342,2171,2156
4,CT010,4836,4800,3832,3779,9310,9159,7094,6949,3205,...,24883,24689,23551,23497,21469,21299,5055,4960,1849,1832
5,CT012,5770,5692,3867,3881,11473,11151,6220,6301,3977,...,22114,21656,27877,27401,21436,20955,5881,6043,2177,2111
6,CT013,5303,5376,4242,4109,12054,11314,7149,6969,3552,...,29462,25944,26033,24740,20920,19581,5584,5218,2225,2250
7,NT015,4317,4277,3245,3133,12903,12628,8121,8150,2851,...,23412,23971,23097,23044,19913,19816,4607,4331,1672,1651
8,NT016,6578,6296,4624,4526,11288,10542,6731,6581,4024,...,32128,30323,25880,24787,25579,24179,4570,4612,2750,2666
9,NT017,6702,6470,3428,3140,14781,12194,5846,5542,3674,...,22892,20276,25252,23227,23837,20688,4752,4515,2171,2074


In [24]:


# ---------- CONFIG ----------
IN_CSV   = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_lh_rh.csv")
MAP_XLSX = Path("longitude_stats_without_normalizations/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_lobes_lr.csv")

AGGREGATION = "sum"   # "sum" (default) or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]
LOBEFILTER_COL_CANDIDATES = ["Lobes"]
# --------------------------------

pat = re.compile(r"^(?P<struct>.+)_(?P<hemi>lh|rh)_ses(?P<ses>\d+)$", re.IGNORECASE)


# --- 1) Load subject table ---
df = pd.read_csv(IN_CSV)

# Identify all columns that look like <struct>_(lh|rh)_ses<1|2>
lr_cols = [c for c in df.columns if pat.match(c)]

# Collect unique sessions present (e.g., {1,2})
sessions_present = sorted({int(pat.match(c).group("ses")) for c in lr_cols})

# --- 2) Load mapping and build struct -> lobe mapping ---
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)

col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)  # prefer lobefilter

if not col_struct or not col_lobe:
    raise ValueError(f"Could not find mapping columns. StructName={col_struct}, LobeFilter={col_lobe}")

# Normalize mapping keys with slugify so they match the struct part in your IN_CSV columns
# (Your IN_CSV struct segments are typically slugified already from earlier steps.)
map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)

struct_to_lobe = {slugify(s): slugify(l) for s, l in zip(map_df[col_struct], map_df[col_lobe])}

# --- 3) Build lobe-level columns by hemisphere & session ---
# For each lobe, hemi, session → sum values across mapped struct columns
out = df[["subject_code"]].copy()

# Find all struct keys present in the IN_CSV (to intersect with mapping)
present_structs = {}
for c in lr_cols:
    m = pat.match(c)
    present_structs.setdefault(m.group("struct"), set()).add((m.group("hemi").lower(), int(m.group("ses"))))

# All lobes that appear in the mapping for structs actually present in the data
lobes_present = sorted({struct_to_lobe.get(s) for s in present_structs.keys() if struct_to_lobe.get(s)})

# Build columns
for lobe in lobes_present:
    for ses in sessions_present:
        for hemi in ("lh", "rh"):
            # Collect all struct columns that map to this lobe & hemi & session
            cols_for_group = []
            for struct in present_structs.keys():
                if struct_to_lobe.get(struct) != lobe:
                    continue
                col_name = f"{struct}_{hemi}_ses{ses}"
                if col_name in df.columns:
                    cols_for_group.append(col_name)

            new_col = f"{lobe}_{hemi}_ses{ses}"
            out[new_col] = reduce_cols(df, cols_for_group, how=AGGREGATION)


out = out[sorted(out.columns, key=order_key_lobe)]

# --- 5) Save ---
OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level file: longitude_stats_without_normalizations\2_sessions_aparc_lobes_lr.csv


,subject_code,cingulate_lh_ses1,cingulate_rh_ses1,cingulate_lh_ses2,cingulate_rh_ses2,frontal_lh_ses1,frontal_rh_ses1,frontal_lh_ses2,frontal_rh_ses2,insula_lh_ses1,...,occipital_lh_ses2,occipital_rh_ses2,parietal_lh_ses1,parietal_rh_ses1,parietal_lh_ses2,parietal_rh_ses2,temporal_lh_ses1,temporal_rh_ses1,temporal_lh_ses2,temporal_rh_ses2
0,NT002,9706,9865,9481,9582,89293,84816,85423,79681,6489,...,25788,27386,58614,59527,56719,57026,54678,55487,53217,54179
1,CT003,10398,10116,10359,10018,93810,93950,91608,91864,6930,...,25565,26985,67272,69244,65684,67453,61560,58386,60339,56304
2,NT005,7537,9053,7162,8678,88186,83363,80631,77493,8207,...,21904,22303,54673,57590,50169,52917,56064,52803,54508,51899
3,CT007,10191,11449,10221,11814,92583,89581,95697,90855,7518,...,25342,26156,51499,56189,52942,56020,56565,54945,57935,54860
4,CT010,9747,10332,9666,10275,83272,90581,80950,87320,6532,...,25880,27921,60122,46754,59095,47148,52625,52191,51868,51920
5,CT012,10343,10517,10094,10342,91091,86217,87877,84407,7779,...,24029,25579,54042,53904,52373,53256,63107,62111,61617,61001
6,CT013,11247,10899,10874,10164,94079,95972,89250,89608,7357,...,28401,28536,60409,62153,55750,56117,60272,57124,59441,56059
7,NT015,10087,7667,9844,7592,78371,79233,77450,77730,6872,...,27463,27919,52742,53544,53269,53934,54022,52978,53207,51489
8,NT016,9797,11546,9602,11334,86041,84369,81924,81611,6917,...,23785,27984,63597,61607,59910,58494,60854,57609,58983,56324
9,NT017,9958,8556,9122,8020,89886,100880,78901,87083,7106,...,21435,21994,63865,54713,56443,47747,56102,54743,54436,52294


In [25]:

# ---------- CONFIG ----------
IN_CSV   = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_merge.csv")  # use your current file
MAP_XLSX = Path("longitude_stats_without_normalizations/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_lobes_full_brain.csv")

AGGREGATION = "sum"   # "sum" (default) or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]  # mapping struct name column
LOBEFILTER_COL_CANDIDATES = ["Lobes"]      # mapping lobe column
# --------------------------------

# Patterns (no helper re-definitions)
pat_nohemi = re.compile(r"^(?P<struct>.+)_ses(?P<ses>\d+)$", re.IGNORECASE)

# 1) Load input table
df = pd.read_csv(IN_CSV)

# already merged per-structure (no hemisphere)
nohemi_cols = [c for c in df.columns if pat_nohemi.match(c)]
df_struct = df[["subject_code"] + nohemi_cols].copy()

# 3) Sessions present
sessions_present = sorted({int(pat_nohemi.match(c).group("ses")) for c in df_struct.columns if pat_nohemi.match(c)})

# 4) Load mapping and build struct→lobe dict using your pick_column() + slugify()
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)
col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_lobe   = pick_column(map_df, LOBEFILTER_COL_CANDIDATES)
if not col_struct or not col_lobe:
    raise ValueError(f"Mapping columns not found. StructName={col_struct}, Lobes={col_lobe}")

map_df = map_df[[col_struct, col_lobe]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_lobe]   = map_df[col_lobe].astype(str)
struct_to_lobe = {slugify(s): slugify(l) for s, l in zip(map_df[col_struct], map_df[col_lobe])}

# Which structs exist in the input?
present_structs = {pat_nohemi.match(c).group("struct") for c in df_struct.columns if pat_nohemi.match(c)}
lobes_present = sorted({struct_to_lobe.get(s) for s in present_structs if struct_to_lobe.get(s)})

# 5) Build lobe totals per session (no hemisphere)
out = df_struct[["subject_code"]].copy()
for lobe in lobes_present:
    for ses in sessions_present:
        cols_for_group = []
        for struct in present_structs:
            if struct_to_lobe.get(struct) != lobe:
                continue
            col_name = f"{struct}_ses{ses}"
            if col_name in df_struct.columns:
                cols_for_group.append(col_name)
        out[f"{lobe}_ses{ses}"] = reduce_cols(df_struct, cols_for_group, how=AGGREGATION)

# 6) Order & save using your order_key_lobe()
out = out[sorted(out.columns, key=order_key_lobe)]

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level (no hemisphere) file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level (no hemisphere) file: longitude_stats_without_normalizations\2_sessions_aparc_lobes_full_brain.csv


,subject_code,cingulate_ses1,cingulate_ses2,frontal_ses1,frontal_ses2,insula_ses1,insula_ses2,occipital_ses1,occipital_ses2,parietal_ses1,parietal_ses2,temporal_ses1,temporal_ses2
0,NT002,19571,19063,174109,165104,13009,12663,54898,53174,118141,113745,110165,107396
1,CT003,20514,20377,187760,183472,13341,13115,53277,52550,136516,133137,119946,116643
2,NT005,16590,15840,171549,158124,15975,15528,45661,44207,112263,103086,108867,106407
3,CT007,21640,22035,182164,186552,14860,14758,51376,51498,107688,108962,111510,112795
4,CT010,20079,19941,173853,168270,13295,13054,54790,53801,106876,106243,104816,103788
5,CT012,20860,20436,177308,172284,15635,15424,50069,49608,107946,105629,125218,122618
6,CT013,22146,21038,190051,178858,14546,14032,58386,56937,122562,111867,117396,115500
7,NT015,17754,17436,157604,155180,13580,13352,56130,55382,106286,107203,107000,104696
8,NT016,21343,20936,170410,163535,14350,14202,53424,51769,125204,118404,118463,115307
9,NT017,18514,17142,190766,165984,13953,13094,44332,43429,118578,104190,110845,106730


In [26]:

# ---------- CONFIG ----------
IN_CSV   = Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_merge.csv")  # use your current file
MAP_XLSX = Path("longitude_stats_without_normalizations/mapping_lobes_networks.xlsx")
MAP_SHEET = "aparc_voxels_normalized_ttests"

OUT_CSV = Path("longitude_stats_without_normalizations/2_sessions_aparc_nets_full_brain.csv")

AGGREGATION = "sum"   # "sum" (default) or "mean"
STRUCTNAME_COL_CANDIDATES = ["Parameter"]  # mapping struct name column
YEOFILTER_COL_CANDIDATES = ["Yeo"]      # mapping lobe column
# --------------------------------

# Patterns (no helper re-definitions)
pat_nohemi = re.compile(r"^(?P<struct>.+)_ses(?P<ses>\d+)$", re.IGNORECASE)

# 1) Load input table
df = pd.read_csv(IN_CSV)

# already merged per-structure (no hemisphere)
nohemi_cols = [c for c in df.columns if pat_nohemi.match(c)]
df_struct = df[["subject_code"] + nohemi_cols].copy()

# 3) Sessions present
sessions_present = sorted({int(pat_nohemi.match(c).group("ses")) for c in df_struct.columns if pat_nohemi.match(c)})

# 4) Load mapping and build struct→lobe dict using your pick_column() + slugify()
map_df = pd.read_excel(MAP_XLSX, sheet_name=MAP_SHEET)
col_struct = pick_column(map_df, STRUCTNAME_COL_CANDIDATES)
col_net   = pick_column(map_df, YEOFILTER_COL_CANDIDATES)
if not col_struct or not col_net:
    raise ValueError(f"Mapping columns not found. StructName={col_struct}, net={col_net}")

map_df = map_df[[col_struct, col_net]].dropna()
map_df[col_struct] = map_df[col_struct].astype(str)
map_df[col_net]   = map_df[col_net].astype(str)
struct_to_net = {slugify(s): slugify(l) for s, l in zip(map_df[col_struct], map_df[col_net])}

# Which structs exist in the input?
present_structs = {pat_nohemi.match(c).group("struct") for c in df_struct.columns if pat_nohemi.match(c)}
networks_present = sorted({struct_to_net.get(s) for s in present_structs if struct_to_net.get(s)})

# 5) Build lobe totals per session (no hemisphere)
out = df_struct[["subject_code"]].copy()
for net in networks_present:
    for ses in sessions_present:
        cols_for_group = []
        for struct in present_structs:
            if struct_to_net.get(struct) != net:
                continue
            col_name = f"{struct}_ses{ses}"
            if col_name in df_struct.columns:
                cols_for_group.append(col_name)
        out[f"{net}_ses{ses}"] = reduce_cols(df_struct, cols_for_group, how=AGGREGATION)


# 6) Order & save using your order_key_lobe()
out = out[sorted(out.columns, key=order_key_lobe)]

# Rename network columns to your desired labels
net_map = {
    "default_mode_ses1"        : "DMN_GM_ses1",
    "default_mode_ses2"        : "DMN_GM_ses2",
    "dorsal_attention_ses1"    : "DAN_GM_ses1",
    "dorsal_attention_ses2"    : "DAN_GM_ses2",   # <- fixed
    "frontoparietal_ses1"      : "FPN_GM_ses1",
    "frontoparietal_ses2"      : "FPN_GM_ses2",
    "limbic_ses1"              : "LIM_GM_ses1",
    "limbic_ses2"              : "LIM_GM_ses2",
    "somatomotor_ses1"         : "SMN_GM_ses1",
    "somatomotor_ses2"         : "SMN_GM_ses2",
    "ventral_attention_ses1"   : "VAN_GM_ses1",
    "ventral_attention_ses2"   : "VAN_GM_ses2",
    "visual_ses1"              : "VIS_GM_ses1",
    "visual_ses2"              : "VIS_GM_ses2",
}

# Correct signature:
out = out.rename(columns=net_map)

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
out.to_csv(OUT_CSV, index=False)

print(f"Saved lobe-level (no hemisphere) file: {OUT_CSV}")
display(out.head(10))


Saved lobe-level (no hemisphere) file: longitude_stats_without_normalizations\2_sessions_aparc_nets_full_brain.csv


,subject_code,DMN_GM_ses1,DMN_GM_ses2,DAN_GM_ses1,DAN_GM_ses2,FPN_GM_ses1,FPN_GM_ses2,LIM_GM_ses1,LIM_GM_ses2,SMN_GM_ses1,SMN_GM_ses2,VAN_GM_ses1,VAN_GM_ses2,VIS_GM_ses1,VIS_GM_ses2
0,NT002,145503,140448,29238,28032,45725,43292,61498,59040,79666,76216,53484,51613,74779,72504
1,CT003,163602,159262,32101,31242,49248,47939,68644,67028,90636,89068,52638,51529,74485,73226
2,NT005,140347,130680,28052,25215,41615,38412,60121,58524,78822,73194,57560,54171,64388,62996
3,CT007,145677,147333,25890,26348,45649,47143,59708,61206,82243,83858,58333,59006,71738,71706
4,CT010,145503,143330,24883,24689,45130,43090,55763,54546,71906,70978,56568,55573,73956,72891
5,CT012,152456,148166,22114,21656,44610,43357,64223,62668,81108,79692,59040,57829,73485,72631
6,CT013,161496,151958,29462,25944,46002,44532,65461,64313,82878,75619,59105,56485,80683,79381
7,NT015,136609,135068,23412,23971,42163,41029,57144,55393,73140,74000,51753,50963,74133,72825
8,NT016,151471,144659,32128,30323,42152,40649,62695,61649,78436,74901,62139,59810,74173,72162
9,NT017,156903,139231,22892,20276,49380,43391,61987,59196,79467,69356,62479,55983,63880,63136


In [27]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ========== CONFIG ==========
OUTPUT_XLSX = Path("longitude_stats_without_normalizations/2_sessions_T1_data.xlsx")

# The sheet that merges TWO CSVs into one
MERGED_SHEET_NAME = "global_parameters"
MERGED_CSVS = [
    Path("longitude_stats_without_normalizations/2_sessions_aseg_summary_data.csv"),
    Path("longitude_stats_without_normalizations/2_sessions_aparc_summary_data.csv"),
]
MERGE_KEY = "subject_code"
MERGE_HOW = "outer"           # "inner", "left", "right", "outer"
MERGE_SUFFIXES = ("_x", "_y") # for overlapping column names

# Other individual sheets: sheet_name -> csv path
OTHER_SHEETS = {
    "subcortical_vol": Path("longitude_stats_without_normalizations/2_sessions_aseg_data.csv"),
    "cortical_vol_lr": Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_lh_rh.csv"),
    "cortical_vol_full_brain": Path("longitude_stats_without_normalizations/2_sessions_aparc_voxels_merge.csv"),
    "cortical_lobes_lr_": Path("longitude_stats_without_normalizations/2_sessions_aparc_lobes_lr.csv"),
    "cortical_lobes_full_brain": Path("longitude_stats_without_normalizations/2_sessions_aparc_lobes_full_brain.csv"),  # <- check your path (removed .csv.csv)
    "cortical_networks_full_brain": Path("longitude_stats_without_normalizations/2_sessions_aparc_nets_full_brain.csv"),
}
# ===========================

# ---------- helpers (local to this cell) ----------

# ---------------------------------------------------

# 1) Merge the two CSVs into one sheet
df_a = _read_csv(MERGED_CSVS[0], MERGE_KEY)
df_b = _read_csv(MERGED_CSVS[1], MERGE_KEY)

if df_a.empty and df_b.empty:
    merged_df = pd.DataFrame(columns=[MERGE_KEY])
else:
    if MERGE_KEY not in df_a.columns or MERGE_KEY not in df_b.columns:
        raise ValueError(f"MERGE_KEY '{MERGE_KEY}' must be present in both files.")
    merged_df = df_a.merge(df_b, on=MERGE_KEY, how=MERGE_HOW, suffixes=MERGE_SUFFIXES)

# Keep subject_code first, then sort consistently
merged_df = _order_columns_subject_first(merged_df, MERGE_KEY)
merged_df = _sort_consistent(merged_df, MERGE_KEY)

# 2) Write everything to one Excel workbook (all sheets sorted the same way)
with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
    # merged sheet
    merged_df.to_excel(writer, index=False, sheet_name=MERGED_SHEET_NAME[:31])
    _autosize_columns(writer, merged_df, MERGED_SHEET_NAME[:31])

    # other sheets (one CSV per sheet)
    for sheet_name, csv_path in OTHER_SHEETS.items():
        df = _read_csv(csv_path, MERGE_KEY)
        df = _order_columns_subject_first(df, MERGE_KEY)
        df = _sort_consistent(df, MERGE_KEY)
        safe_name = sheet_name[:31]  # Excel sheet name limit is 31 chars
        df.to_excel(writer, index=False, sheet_name=safe_name)
        _autosize_columns(writer, df, safe_name)

print(f"Saved Excel workbook: {OUTPUT_XLSX}")


Saved Excel workbook: longitude_stats_without_normalizations\2_sessions_T1_data.xlsx


# ssSADDAWWD


## SADA



In [ ]:
DEDWD
